In [1]:
%load_ext autoreload
%autoreload 2

TODO: Update the lin
<a href="https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/endersgame/mean_reversion_attacker/mean_reversion_attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Cover](./docs/cover.jpg)

# En garde, attack!

Welcome to Mid+One! Dive into the world of martingales and market dynamics.

Your challenge: spot tiny shifts in financial time-series, to predict where prices are heading.

It's all about spotting patterns in the elusive mid-price.

# The goal

## Attacking not forecasting!

We don't want to forecast the future prices as this is extremely difficult and requires often a lot of computation. What we want is detect a shift in the market dynamics, up or down, that's it! 

This is a much simpler task and can be done with a simple model. We also need this decision to be computed quickly! Under 20 milliseconds.

To be precise, our attacker will consume a univariate sequence of numerical data points $x_1, x_2, \dots x_t$ and try to exploit deviations from the [martingale property](https://en.wikipedia.org/wiki/Martingale_(probability_theory)), which is to say that we expect the series $x_t$ to satisfy:
$$ E[x_{t+k}] \approx x_t $$
roughly. Of course, there's no such thing in this world as a perfect martingale and it is your job to indicate when
$$ E[x_{t+k}] > x_t + \epsilon $$
by returning a positive value, or conversely. Here $\epsilon$ finds interpretation as a trading cost. The attacker will *typically* return `0` meaning that it thinks:
$$  x_t - \epsilon   > E[x_{t+k}] > x_t + \epsilon $$
because trading opportunities are probably on the rare side - though obviously this is problem dependent. The $\epsilon$ and $k$ (`horizon`) parameters are set [here](https://github.com/microprediction/midone/blob/main/midone/gameconfig.py).

## Scoring

The scoring is straightforward: it corresponds to the profit of a simple trading strategy: if the model predicts a move up over some short horizon (corresponding to a few seconds at most), the strategy is to buy now and sell at the end of the horizon minus some transaction costs.

If we decided to go up at $t$:
$$x_{t+k} - x_t - \epsilon$$
and 
$$x_t - x_{t+k} - \epsilon$$
if we decided to go down at $t$.

The case of a move down is symmetric. In most cases, we don't expect the model to predict a move up or down so we will just do nothing.

This is an example of detecting three move ups (the green bands, detecting down would show as red bands), with two resulting in a profit and one with a loss:

![Profit](./docs/profit.png) 

# The data

This competition is really meant to focus on single streaming series and should be quite agnostic to particular financial instruments. There is a training phase where parameters can be learned on group of similar instruments but overall, we rely on online learning. 

Let's look right away at some data.


# Setup

- Install the crunch CLI
- Use the token to get started with data and submission
- Setup the notebook

In [3]:
%pip install --upgrade crunch-cli -q
!crunch setup --notebook mid-one cobra --token bff0RrUG5HBNHqQYGVmVCrXvWU4V4Ywgk0qHSYPmV6UvewfDPrmN7Im2FF6JDMtR
%pip install --upgrade midone


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

---
Your token seems to have expired or is invalid.

Please follow this link to copy and paste your new setup command:
https://hub.crunchdao.com/competitions/mid-one/submit

If you think that is an error, please contact an administrator.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import crunch
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>


## Load the data

Timeseries come as streams and you can get many streams, split into `train` and `test` datasets. 

A stream is sequence of data points represented by a dictionary. The value of the time series is `pt["x"]` where `p` is the point in the series.

In [5]:
# Load the data
x_train, x_test = crunch.load_streams()

print(f"Loaded {len(x_train)} training streams and {len(x_test)} testing streams")

data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/X_train.parquet (7049425 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/X_test_reduced.parquet (405611 bytes)
data/X_test.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/y_train.parquet (5804278 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/y_test_reduced.parquet (411693 bytes)
data/y_test.parquet: already exists, file length match
data/example_prediction.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/example_prediction_reduc

# Having a peek into the data

`midplot` provides a lot of cool features, one is to able to visualize the data.

Running over the data sequentially like a live algorithm would do is called `replay`.

Let's have a look at the data by replaying the first stream.

In [6]:
from midplot import replay
replay(x_train[:1], with_visualization=True)

Please define the 'infer' function in the main module: for debugging, showing no attacks.


    'data': [{'line': {'color': 'lightgrey'},
              'mode': 'lines',
   …

Note the comment about the `infer` function not being defined. Crunch will pick up automatically your attack function once `infer` is define.

We only ran the first stream for space. Alternatively, you can specify certain streams like this:
```python
replay(x_train, only_stream_ids=[0], with_visualization=True)
```

To only run a subset of the data, you can also specify a `start` and `stop` index:
```python
replay(x_train, only_stream_ids=[0], start_index=0, stop_index=1000, with_visualization=True)
```

### Important flexibility

`midplot` let's you pass regular iterable of float as well so you don't have to recreate these small dictionaries so you can easily run your algorithm on your own data -- very useful for debugging.

This is how you would attack the `log`.

In [7]:
import numpy as np
replay(np.log(range(1, 100)), with_visualization=True)

Please define the 'infer' function in the main module: for debugging, showing no attacks.


    'data': [{'line': {'color': 'lightgrey'},
              'mode': 'lines',
   …

# Let's attack!

Underneath, `crunch` requires an `infer` function. The syntax is not completely trivial so `midplot` provides a helper function to do this.

We only need to define an Attacker class like the other notebooks.

For demo purposes, we will keep a buffer of points and detect a move if the change of price in the second part of the buffer is higher with a threshoold (and will be adjusted) the move in the first part of the buffer.

This is a measure of momentum in some way.

In [8]:
from midone import HORIZON, EPSILON, Attacker
UP, DOWN, NOTHING = 1., -1., 0.
import numpy as np

class MomentumAttacker(Attacker):
    # We will turn this factor into a hyper-parameter!
    factor: float = 2.5
    
    def __init__(self):
        super().__init__()
        self.buffer = []
        self.num_points = 100
    def tick_and_predict(self, x: float, horizon: int = HORIZON) -> float:
        # Add new value and maintain fixed buffer size
        self.buffer.append(x)
        if len(self.buffer) > self.num_points:
            self.buffer.pop(0)  # Remove oldest value

        # Wait until we have enough data
        if len(self.buffer) < self.num_points:
            return NOTHING

        # Split buffer into two halves and calculate change in each half
        mid = self.num_points // 2
        first_half_change = self.buffer[mid - 1] - self.buffer[0]
        second_half_change = self.buffer[-1] - self.buffer[mid]
        if np.sign(first_half_change) != np.sign(second_half_change):
            return NOTHING
        # Compare changes to predict trend
        if np.abs(second_half_change) > self.factor * np.abs(first_half_change):
            return np.sign(second_half_change)
        else:
            return NOTHING
        
from midplot.helpers import wrap
infer = wrap(MomentumAttacker)

This is equivalent to
```python

def infer(
        stream: typing.Iterator[dict],
        hyper_parameters: Any = None,
        with_hyper_parameters_load: bool = False,
        model_directory_path: str = ".",
):
    m = MomentumAttacker()
    yield  # We are ready
    for message in stream:
        pred = m.tick(message["x"], horizon=HORIZON)
        yield pred
        
```

## Note: what are the `hyper_parameters`, `with_hyper_parameters_load` and `model_directory_path` for?

These parameters are important in the training phase where we want to optimize the parameter and save the optimal parameters which we want to load in the inference phase for the actual submission.

It would look like this:
```python
def infer(
        stream: typing.Iterator[dict],
        hyper_parameters: Any = None,
        with_hyper_parameters_load: bool = False,
        model_directory_path: str = ".",
):
    m = MomentumAttacker()
    if hyper_parameters is not None:
        # Defined on your model
        m.update_from_hyper_parameters(hyper_parameters)
    if with_hyper_parameters_load:
        # Load from the params for final inference
        m.load_params(model_directory_path)
    yield  # Leave this here.
    for message in stream:
        pred = m.tick(message["x"], horizon=HORIZON)
        yield pred
```

and the train function would look something like that:
```python

def train(
    streams: typing.List[typing.Iterable[dict]],
    model_directory_path: str
):
    hyper_params = {}
    def optimize(hyper_params):
        res = replay(streams, hyper_parameters=hyper_params)
        return - res.total_score
    # Your optimization function
    hyper_params = optimize(hyper_params)
    # Save the parameters
```



## Let's run the same replay on the first time series

The library will automatically call the `infer` function on the data points.

Let's also display the scoring for this simple algorithm.

In [9]:
replay(x_train[:1], stop_index=500, with_accounting_visualizer=True, with_visualization=True)

HTML(value='\n<style>\n    .table { \n        width: 100%; \n        margin-bottom: 1rem; \n        background…

    'data': [{'line': {'color': 'lightgrey'},
              'mode': 'lines',
   …

## Scenarios

On the last replay, you can click on the graph and will select some points of the time series. This let's you select simple case where you would like your algorithm to pick up a move up or down or do nothing.

This can be very useful to constraints your training to some behavior. Will do some semi-supervised learning in some way.

```python
get_replay_result().save_selected(UP)
```

In [10]:
from midplot import get_replay_result
# Run this to save the scenario
# get_replay_result().save_selected(UP)

In [11]:
from midplot import load_scenarios
movie = load_scenarios()
print(f"Loaded {len(movie.scenarios)} scenarios")

Loaded 1 scenarios




You can run all the scenarios like this:

In [12]:
from midplot.replay import load_scenarios

movie = load_scenarios()

replay(movie.streams(), horizon=HORIZON, with_accounting_visualizer=True, with_visualization=True)

HTML(value='\n<style>\n    .table { \n        width: 100%; \n        margin-bottom: 1rem; \n        background…

    'data': [{'line': {'color': 'lightgrey'},
              'mode': 'lines',
   …

or even check that the scenarios are successful:

In [13]:
def check_scenarios(threshold=None):
    movie = load_scenarios()
    r = replay(movie.streams())
    return  r.check_scenarios(movie.scenarios, threshold=threshold)

ok, checks = check_scenarios(threshold=0.5)
print(f"Failed {len(checks.failed)} Success {len(checks.success)}")

Failed 1 Success 0


## Let's train!

Training here means we will find good values of the parameter of the model, in that case the `factor` parameter.

We will use the `nevergrad` package for optimization as an example.

In [14]:
!pip install nevergrad -q
import nevergrad as ng


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [15]:
import json
from pydantic import BaseModel, Field
    
def save_param(filename: str, factor: float):
        with open(filename, 'w') as f:
            f.write(json.dumps({"factor": factor}))
            
def load_from_param(filename: str) -> MomentumAttacker:
    with open(filename, 'r') as f:
        data = json.load(f)
        factor = float(data["factor"])
        m = MomentumAttacker()
        m.factor = factor
        return m
            
attacker = MomentumAttacker()
attacker.factor = 2.7
save_param('params.json', attacker.factor)
attacker = load_from_param('params.json')
assert attacker.factor == 2.7    

With this save/load mechanism, we can now build a simple optimizer which we will use for the train function.

But first, we need to tweak the `infer` function to load this hyper parameters

In [16]:
from typing import Iterator, Any
def infer(
        stream: Iterator[dict],
        hyper_parameters: Any = None,
        with_hyper_parameters_load: bool = False,
        model_directory_path: str = ".",
):
    m = MomentumAttacker()
    if hyper_parameters is not None:
        m.factor = hyper_parameters 
    yield  # Leave this here.
    for message in stream:
        pred = m.tick_and_predict(message["x"], horizon=HORIZON)
        yield pred

In [17]:
from typing import List, Iterable
import os

def train(
        streams: List[Iterable[dict]],
        model_directory_path: str = "."):
    factor = ng.p.Scalar(init=1.25, lower=1., upper=3.)
    parametrization = ng.p.Instrumentation(factor)
    optimizer = ng.optimizers.OnePlusOne(parametrization=parametrization, budget=10)

    # Define the objective function that we want to minimize
    def objective(factor_value):
        # To go faster, only subset of data
        r = replay(streams, only_stream_ids=[1, 2], stop_index=500, hyper_parameters=factor_value)
        print(f"Score with factor={factor_value} -> {r.total_score}")
        return - r.total_score

    # Run the optimization
    recommendation = optimizer.minimize(objective)
    # Get the optimal factor value
    optimal_factor = recommendation[0].value[0]
    print("Optimal factor", optimal_factor)
    save_param(os.path.join(model_directory_path, "params.json"), optimal_factor)
    
# We run the optimizer
train(x_train)

Score with factor=1.25 -> 32.50307692310352
Score with factor=1.052143659185795 -> 37.743269230794986
Score with factor=2.268618244160313 -> 29.022692307707842
Score with factor=1.6389814731428825 -> 21.834807692328546
Score with factor=1.8925258161805205 -> 20.69000000001092
Score with factor=2.0286721014347027 -> 25.012692307706025
Score with factor=1.7075720729998114 -> 28.549615384629803
Score with factor=1.0501602323044557 -> 37.743269230794986
Score with factor=1.7013677838452588 -> 27.777884615399316
Score with factor=1.18856787696814 -> 28.793269230793165
Optimal factor 1.0501602323044557


Now we can finish the proper version of `infer` by adding the loading:

In [30]:
def infer(
        stream: Iterator[dict],
        hyper_parameters: Any = None,
        with_hyper_parameters_load: bool = False,
        model_directory_path: str = ".",
):
    m = MomentumAttacker()
    
    if with_hyper_parameters_load:
        m = load_from_param(os.path.join(model_directory_path, "params.json"))

    yield  # Leave this here.
    for message in stream:
        pred = m.tick_and_predict(message["x"], horizon=HORIZON)
        yield pred
        
replay(streams=x_train, only_stream_ids=[5], stop_index=1000, with_visualization=True, with_accounting_visualizer=True)

HTML(value='\n<style>\n    .table { \n        width: 100%; \n        margin-bottom: 1rem; \n        background…

    'data': [{'line': {'color': 'lightgrey'},
              'mode': 'lines',
   …

### And we are done!

## Scoring double-check

Let's double check that the scoring is consistent between the attacker and the backend.

For that, we will use the `crunch.test()` and use the testing function written that mimicks the backend computation.

In [31]:
from midplot import replay

res = replay(x_test, horizon=HORIZON, epsilon=EPSILON, with_hyper_parameters_load=False)

In [32]:
from midplot.test import  pnl
def train():
    pass
predictions = crunch.test()

18:14:35 no forbidden library found
18:14:35 
18:14:35 started
18:14:35 running local test
18:14:35 internet access isn't restricted, no check will be done
18:14:35 
18:14:39 starting stream loop...


data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/X_train.parquet (7049425 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/X_test_reduced.parquet (405611 bytes)
data/X_test.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/y_train.parquet (5804278 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/y_test_reduced.parquet (411693 bytes)
data/y_test.parquet: already exists, file length match
data/example_prediction.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/78/example_prediction_reduc

18:14:39 call: train - stream.len=319
18:14:39 looping stream=`stream_0` (1/20)
18:14:39 call: infer (1/1)
18:14:40 looping stream=`stream_1` (2/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_10` (3/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_11` (4/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_12` (5/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_13` (6/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_14` (7/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_15` (8/20)
18:14:40 call: infer (1/1)
18:14:40 looping stream=`stream_16` (9/20)
18:14:40 call: infer (1/1)
18:14:41 looping stream=`stream_17` (10/20)
18:14:41 call: infer (1/1)
18:14:41 looping stream=`stream_18` (11/20)
18:14:41 call: infer (1/1)
18:14:41 looping stream=`stream_19` (12/20)
18:14:41 call: infer (1/1)
18:14:41 looping stream=`stream_2` (13/20)
18:14:41 call: infer (1/1)
18:14:41 looping stream=`stream_3` (14/20)
18:14:41

In [33]:
res_check = pnl(predictions, x_test, HORIZON, EPSILON)
print(f"With PnL: {res.total_score} and with the test function: {res_check.total_score}")

With PnL: -146.31970620566835 and with the test function: -146.58720620566876
